# Gray-Scott f-k Parameter Map Visualization

Interactive visualization of Gray-Scott reaction-diffusion patterns.
Click on any point in the f-k map to view the corresponding GIF animation.

## Code

In [8]:
# Enable interactive backend for pick events
try:
    %matplotlib widget
    print('Using %matplotlib widget')
except Exception:
    try:
        %matplotlib notebook
        print('Using %matplotlib notebook')
    except Exception:
        import matplotlib
        print('Interactive backend not available. Current backend:', matplotlib.get_backend())
        print('Install ipympl: pip install ipympl, then restart the kernel.')

Using %matplotlib widget


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import ipywidgets as widgets
import os

# Import our analysis library
import fk_analysis

In [10]:
# Load data (uses cache if available, otherwise processes all GIF files)
# First run will take several minutes to process all GIFs
# Subsequent runs will load from cache instantly
data = fk_analysis.load_or_process_data(gif_dir='gif', use_cache=True, verbose=True)

f_values = data['f_values']
k_values = data['k_values']
variations = data['variations']
avail_paths = data['paths']

Cache loaded from fk_data_cache.npz
  15600 data points


In [11]:
# Create GIF preview widgets (128x128 to match actual GIF size)
output_box = widgets.Output(layout=widgets.Layout(
    width='128px', height='128px', overflow='hidden', 
    border='1px solid #ccc', flex='0 0 auto', 
    padding='0px', margin='0px'
))

filename_label = widgets.HTML(
    value='<div style="width:128px; word-wrap:break-word; font-size:12px; text-align:center;">'
          'Click a point to preview</div>'
)

gif_container = widgets.VBox(
    [filename_label, output_box],
    layout=widgets.Layout(gap='4px')
)

In [ ]:
# Build interactive f-k map
plt.ioff()
fig, ax = plt.subplots(figsize=(6, 5))

# Scatter plot with spatial variation as color
sc = ax.scatter(k_values, f_values, c=variations, s=24, alpha=0.7, 
                picker=5, cmap='viridis')
# Color bar removed

ax.set_xlabel('k')
ax.set_ylabel('f')
ax.set_title('f-k Map: Spatial Variation (click to view GIF)')
ax.grid(True, alpha=0.3)

# Add theoretical bifurcation curves
kmin, kmax = k_values.min(), k_values.max()
fmin, fmax = f_values.min(), f_values.max()

def plot_curve_simple(G, klim, flim, ax, color='crimson', linewidth=2, label=None, n=400):
    k = np.linspace(klim[0], klim[1], n)
    f = np.linspace(flim[0], flim[1], n)
    K, F = np.meshgrid(k, f)
    Z = G(F, K)
    if np.iscomplexobj(Z):
        Z = np.where(np.abs(np.imag(Z)) < 1e-10, np.real(Z), np.nan)
    else:
        Z = np.asarray(Z, dtype=float)
    ax.contour(K, F, Z, levels=[0.0], colors=color, linewidths=linewidth, alpha=0.8)
    if label:
        ax.plot([], [], color=color, linewidth=linewidth, label=label)

# Hopf bifurcation curve
def G_Hopf(f, k):
    return 4*k - (f + np.sqrt(f*(f-4*(f+k)**2)))**2 / ((f + k)**2)

plot_curve_simple(G_Hopf, (kmin, kmax), (fmin, fmax), ax, 
                  color='green', linewidth=1.5, label='Hopf')

# Numerical solution boundary
def G_NumSoln(f, k):
    return f - 4*f**2 - 8*f*k - 4*k**2

plot_curve_simple(G_NumSoln, (kmin, kmax), (fmin, fmax), ax, 
                  color='red', linewidth=1.5, label='NumSoln')

ax.legend(loc='upper right', fontsize=9)

# Set aspect ratio
data_range_k = k_values.max() - k_values.min()
data_range_f = f_values.max() - f_values.min()
ax.set_aspect(data_range_k / (1.2 * data_range_f), adjustable='box')

fig.tight_layout(pad=2.0)

last_annotation = None

# Click event handler
def on_pick(event):
    global last_annotation
    if last_annotation is not None:
        last_annotation.remove()
        last_annotation = None

    ind = event.ind
    if len(ind) == 0:
        return
    idx = ind[0]
    f = float(f_values[idx])
    k = float(k_values[idx])
    var = float(variations[idx])

    # Annotate clicked point
    last_annotation = ax.annotate(
        f"f={f:.4f}, k={k:.4f}\\nvar={var:.1f}", (k, f),
        textcoords="offset points", xytext=(10, 10),
        bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.6)
    )
    fig.canvas.draw_idle()

    # Find and display GIF
    gif_path = fk_analysis.find_gif_for_fk(f, k, f_values, k_values, avail_paths)
    if not gif_path or not os.path.exists(gif_path):
        print('GIF not found near', (f, k))
        return

    print('Displaying:', gif_path)
    
    # Update filename label
    filename = os.path.basename(gif_path)
    filename_label.value = (
        f'<div style="width:128px; word-wrap:break-word; font-size:12px; text-align:center;">'
        f'{filename}</div>'
    )
    
    # Display GIF at actual size (128x128) with no padding
    with output_box:
        clear_output(wait=True)
        # Read GIF file and encode as base64 to avoid path issues
        import base64
        with open(gif_path, 'rb') as f:
            gif_data = base64.b64encode(f.read()).decode('utf-8')
        # Use HTML img tag with inline style to remove all padding/margin
        from IPython.display import HTML
        display(HTML(f'<img src="data:image/gif;base64,{gif_data}" width="128" height="128" style="display:block; margin:0; padding:0; border:none; line-height:0;" />'))

# Configure canvas and display
fig.canvas.layout = widgets.Layout(
    width='600px', height='500px', 
    border='1px solid #ccc', flex='0 0 auto'
)

container = widgets.HBox(
    [fig.canvas, gif_container],
    layout=widgets.Layout(
        align_items='center', 
        justify_content='flex-start', 
        gap='12px', 
        overflow='visible'
    )
)

clear_output(wait=False)
display(container)

cid = fig.canvas.mpl_connect('pick_event', on_pick)